In [1]:
import imageio
from matplotlib import pyplot as plt
from JPEG import JPEG
from libs.JPEGStandardHuffmanTable import STD_HUFTAB_LUMIN_DC, STD_HUFTAB_CHROMIN_DC, STD_HUFTAB_LUMIN_AC, STD_HUFTAB_CHROMIN_AC

In [15]:

Image_set = dict(
    cover = "../_Dataset/Lenna/lena_color.gif"
    # gray = "./test_gray.jpg"
)

Images_name = []
Images = []
Images_path = []
Images_len = 0
for name, path in Image_set.items():
    Images_name.append(name)
    Images.append(imageio.imread(path))
    Images_path.append(path)
    Images_len += 1

jpeg = JPEG(image_path = Images_path[0])
jpeg.useStandardHuffmanTable()
# jpeg.useDefineTable()
# jpeg.HuffmanTableTypeIndex = 2
log = dict()
jpeg.encode(quality=70, isColor=True, save_path="./test_gray", finished_info=log)
log, jpeg.HuffmanTableTypeIndex

C:\Users\BREND\AppData\Local\Temp\ipykernel_14496\3474917347.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  Images.append(imageio.imread(path))


({'date': 1661849750.1377437,
  'file_path': 'test_gray.jpg',
  'size': (512, 512),
  'quality': 70,
  'file_size': 44471,
  'compression_ratio': '511.20%',
  'run_time': 1.3125677108764648,
  'PSNR': 33.27194616308717},
 0)

## Deqantized

## Quantized for RDH

In [3]:
import numpy as np
quality = 70

jpeg = JPEG(image_path = Images_path[0])
jpeg.quality = quality
jpeg.useStandardHuffmanTable()
jpeg.block_size = 8

jpeg._img2ycbcr(isColor=False)
jpeg._ycbcrDCT()

jpeg.QuantizationTable = jpeg.quantizedTable(jpeg.quality)
quant = np.zeros(jpeg.ycbcr.shape, dtype=jpeg.fileDtype)
quantized = jpeg.dct[:, :, 0]/np.tile(jpeg.QuantizationTable[jpeg.color_components[0]["Quantization_table_index"]], (jpeg.height//8, jpeg.width//8))
quantized_around = np.around(quantized)

quantized, quantized_around

(array([[ 2.33330000e+01,  1.24147651e+00,  3.56126619e-01, ...,
         -2.93213927e-01, -3.00654210e-01, -1.03900037e-01],
        [ 1.23495295e+00, -1.67304618e-01, -1.87568039e-01, ...,
         -2.02550655e-01, -8.47239535e-02, -1.74568428e-01],
        [-5.37635795e-01,  1.47120839e-01,  2.07073519e-02, ...,
          1.10077788e-01,  3.74437445e-03,  7.87036117e-02],
        ...,
        [-1.64592765e-01, -5.65239852e-02,  3.57404164e-01, ...,
          7.39995127e-02, -1.06363426e-02, -5.76848729e-02],
        [ 1.80716958e-01, -4.72414994e-02, -1.12246106e-01, ...,
          1.39564076e-04,  2.01836269e-02,  2.10801282e-03],
        [-5.72125805e-02, -4.98070924e-02, -1.48639283e-02, ...,
         -8.86287379e-02,  8.19375197e-02,  1.40014514e-02]]),
 array([[23.,  1.,  0., ..., -0., -0., -0.],
        [ 1., -0., -0., ..., -0., -0., -0.],
        [-1.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [-0., -0.,  0., ...,  0., -0., -0.],
        [ 0., -0., -0., ...,  0.,  0

In [4]:
quantized_around_DC = np.zeros((quantized_around.shape[0], quantized_around.shape[1]), dtype=jpeg.fileDtype)
for i in range(0, quantized_around_DC.shape[0], jpeg.block_size):
    for j in range(0, quantized_around_DC.shape[1], jpeg.block_size):
        quantized_around_DC[i, j] = quantized_around[i, j]

quantized_around_AC = quantized_around - quantized_around_DC

quantized_around_DC, quantized_around_AC

(array([[23,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]]),
 array([[ 0.,  1.,  0., ..., -0., -0., -0.],
        [ 1., -0., -0., ..., -0., -0., -0.],
        [-1.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [-0., -0.,  0., ...,  0., -0., -0.],
        [ 0., -0., -0., ...,  0.,  0.,  0.],
        [-0., -0., -0., ..., -0.,  0.,  0.]]))

In [5]:
quantized[abs(quantized_around_AC)==2]

array([-2.04643844, -1.79736692, -1.53269884, ..., -2.16115964,
       -1.60014385,  1.63298864])

In [6]:
posi_value = quantized[abs(quantized_around_AC)==2]>0
sym = np.zeros((len(posi_value)), dtype=np.int8)
sym[posi_value==False] = -1
sym[posi_value==True] = 1
sym

array([-1, -1, -1, ..., -1, -1,  1], dtype=int8)

In [7]:
shuffledBlocks = quantized_around_AC[abs(quantized_around_AC)==2]
shuffledBlocks, quantized[abs(quantized_around_AC)==2]

(array([-2., -2., -2., ..., -2., -2.,  2.]),
 array([-2.04643844, -1.79736692, -1.53269884, ..., -2.16115964,
        -1.60014385,  1.63298864]))

In [8]:
posi_value = shuffledBlocks[abs(quantized[abs(quantized_around_AC)==2])>2]>0
sym = np.zeros((len(posi_value)), dtype=np.int8)
sym[posi_value==False] = -1
sym[posi_value==True] = 1
# shuffledBlocks, sym
shuffledBlocks[abs(quantized[abs(quantized_around_AC)==2])>2] = 3*sym
shuffledBlocks

array([-3., -2., -2., ..., -3., -2.,  2.])

In [9]:
posi_value = shuffledBlocks[abs(quantized[abs(quantized_around_AC)==2])<=2]>0
sym = np.zeros((len(posi_value)), dtype=np.int8)
sym[posi_value==False] = -1
sym[posi_value==True] = 1
# shuffledBlocks, sym
shuffledBlocks[abs(quantized[abs(quantized_around_AC)==2])<=2] = 1*sym
shuffledBlocks

array([-3., -1., -1., ..., -3., -1.,  1.])

In [10]:
quantized_around_AC[abs(quantized_around_AC)==2] = shuffledBlocks

quantized_around_AC[abs(quantized_around_AC) == 2]

array([], dtype=float64)

In [11]:
quantized_around = quantized_around_DC + quantized_around_AC
quantized_around

array([[23.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [-1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
jpeg.quant = np.zeros(jpeg.ycbcr.shape, dtype=jpeg.fileDtype)

jpeg.quant[:, :, 0] = quantized_around

In [13]:
jpeg._ycbcrZigzag()
jpeg._DPCMandRLE()
jpeg.HuffmanTableBuilding(HuffmanTableTypeIndex=jpeg.HuffmanTableTypeIndex)
jpeg._HuffmanEncoding()
jpeg._EntropyCodedStgment()
jpeg._writeToFile(save_path="./Lena_Q{}_M({})".format(quality, "RDH Shuffled"))

True

In [14]:
jpeg.analize()

{'date': 1661841900.4962435,
 'file_path': 'Lena_Q70_MRDH Shuffled.jpg',
 'size': (512, 512),
 'quality': 70,
 'file_size': 32127,
 'compression_ratio': '707.61%',
 'PSNR': 15.73138103082989}

## HuffmanTree

In [8]:
from libs.HuffmanTree import LimitedLenHuffmanTree

# freq_table = ["A"]*14+["B"]*6+["C"]*3+["D"]*2
freq_table = "a"*16+"b"*8+"c"*19+"d"*32+"e"*21+"f"*10+"g"*5

HuffmanTree = LimitedLenHuffmanTree.init_arr(freq_table)
HuffmanTable = HuffmanTree.create_huffman_table()
HuffmanTable

{'d': '00',
 'e': '010',
 'c': '011',
 'a': '100',
 'f': '101',
 'b': '110',
 'g': '1110'}

{'A': '00', 'B': '01', 'C': '10', 'D': '110'}